# ChatTool 基础使用指南

本示例介绍 ChatTool 的三个核心功能：
1. 配置密钥
2. 基础对话
3. 批量对话

## 1. 配置密钥

ChatTool 提供了统一的配置管理接口 `OpenAIConfig`。你可以通过它查看或修改 API 配置。

In [ ]:
from chattool import debug_log
debug_log()

In [3]:
from chattool import OpenAIConfig, Chat

# 查看当前配置（自动从环境变量或 .env 文件读取）
print(f"Current API Key: {OpenAIConfig.OPENAI_API_KEY.mask_value()}")
print(f"Current Base URL: {OpenAIConfig.OPENAI_API_BASE.value}")
print(f"Current Base URL: {OpenAIConfig.OPENAI_API_MODEL.value}")

# 如果需要动态修改配置
# OpenAIConfig.OPENAI_API_KEY.value = "sk-new-key..."
# OpenAIConfig.OPENAI_API_BASE.value = "https://api.openai-proxy.com/v1"
# OpenAIConfig.OPENAI_API_MODEL.value = "gpt-3.5-turbo"

Current API Key: sk-n3Qws***************************************7e8f2073
Current Base URL: https://4.0.wokaai.com/v1
Current Base URL: gpt-3.5-turbo


## 2. 基础对话

使用 `Chat` 对象进行单轮或多轮对话。

In [5]:
# 初始化 Chat 对象
chat = Chat(model="gpt-3.5-turbo")

# 1. 简单问答 (不记录历史)
answer = chat.ask("1+1=?", update_history=False)
print(f"简单问答: {answer}")

# 2. 多轮对话
chat.system("你是一个数学助手")
chat.user("斐波那契数列的前3个数是？")
resp = chat.get_response()
print(f"\n用户: {chat.chat_log[-2]['content']}")
print(f"助手: {resp.content}")

# 继续对话
chat.user("那下一个数呢？")
resp = chat.get_response()
print(f"\n用户: {chat.chat_log[-2]['content']}")
print(f"助手: {resp.content}")

简单问答: 1 + 1 = 2

用户: 斐波那契数列的前3个数是？
助手: 斐波那契数列的前3个数是：1、1、2。

用户: 那下一个数呢？
助手: 斐波那契数列的每个数都是前两个数的和。前3个数是1、1、2，所以下一个数是1 + 2 = 3。


## 3. 批量对话

使用 `batch_process_chat` 并发处理多个对话请求，适合数据标注场景。

In [ ]:
# 1. 定义处理函数
async def process_text(text):
    chat = Chat()
    # 可以在这里添加 system prompt
    chat.system("请将这句话翻译成英文")
    await chat.async_ask(text)
    return chat

# 2. 准备数据
texts = [
    "你好，世界",
    "今天天气不错",
    "人工智能正在改变生活"
]

# 3. 批量处理 (nworker 控制并发数)
chats = Chat.batch_process_chat(
    texts,
    process_text,
    nworker=3,
    checkpoint='chat_example.jsonl' # 用于保存中间结果，方便中断后处理
)

# 4. 查看结果
for i, chat in enumerate(chats):
    print(f"原文: {texts[i]}")
    print(f"译文: {chat.last_message}")
    print("-" * 20)

100%|██████████| 3/3 [00:01<00:00,  2.61it/s]

原文: 你好，世界
译文: Hello, world.
--------------------
原文: 今天天气不错
译文: The weather is nice today.
--------------------
原文: 人工智能正在改变生活
译文: Artificial intelligence is changing life.
--------------------
